In [2]:
import pandas as pd
import os

#### Procedimiento para hacer ETL con los datos de business de yelp con la cuidad y estado corregidos

In [33]:
df_business = pd.read_parquet("D:/PROYECTO FINAL/Yelp/business2.parquet")

In [13]:
df_business['categories'] = df_business['categories'].str.split(',')
df_business= df_business.dropna(subset=['categories']) # Elimina datos nulos de la columna

In [14]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15276 entries, 0 to 15275
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   business_id    15276 non-null  object 
 1   name           15276 non-null  object 
 2   address        15190 non-null  object 
 3   city           15276 non-null  object 
 4   state          15276 non-null  object 
 5   postal_code    15272 non-null  object 
 6   latitude       15276 non-null  float64
 7   longitude      15276 non-null  float64
 8   stars          15276 non-null  float64
 9   review_count   15276 non-null  int64  
 10  is_open        15276 non-null  int64  
 11  hours          12541 non-null  object 
 12  Es_Restaurant  15276 non-null  bool   
 13  ciudad         15168 non-null  object 
 14  estado         15276 non-null  object 
 15  attributes     15101 non-null  object 
 16  categories     15276 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(11)
memor

In [15]:
df_business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,hours,Es_Restaurant,ciudad,estado,attributes,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '...",True,Philadelphia,Pennsylvania,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Food, Bubble Tea, Coffee & Te..."
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,AB,89502,39.476117,-119.789339,2.5,339,1,"{'Friday': '11:0-21:0', 'Monday': '11:0-22:0',...",True,Reno,Nevada,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Italian]"
2,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,Nashville,TN,37201,36.159886,-86.773197,4.0,161,0,"{'Friday': '16:0-23:0', 'Monday': None, 'Satur...",True,Nashville,Tennessee,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Japanese, Seafood]"


In [16]:
df_business.shape

(15276, 17)

In [17]:
df_business['estado']= df_business['estado'].str.strip()  # quita los espacios vacios

In [18]:
lista_estados= ['Florida', 'Pennsylvania', 'Tennessee', 'California', 'Texas', 'New York']
df_business= df_business[df_business['estado'].isin(lista_estados)]

In [19]:
df_business.shape

(7879, 17)

In [20]:
df_business['estado'].value_counts()

estado
Pennsylvania    3823
Florida         2509
Tennessee       1181
California       366
Name: count, dtype: int64

In [21]:
df_category= df_business[['business_id','categories']]
df_category= df_category.explode('categories')
df_category= df_category.dropna(subset=['categories']) # Elimina datos nulos de la columna
df_category['categories']= df_category['categories'].str.strip()    # Elimina los espacios

In [22]:
df_category= df_category.rename(columns={'categories': 'category_name'}) # cambiar nombre de las columnas
df_category['platform']= 2

In [23]:
df_category.head(5)

,business_id,category_name
0,MTSW4McQd7CbVtyjqoe9mw,Restaurants
0,MTSW4McQd7CbVtyjqoe9mw,Food
0,MTSW4McQd7CbVtyjqoe9mw,Bubble Tea
0,MTSW4McQd7CbVtyjqoe9mw,Coffee & Tea
0,MTSW4McQd7CbVtyjqoe9mw,Bakeries


In [24]:
df_atributes= df_business[['business_id','attributes']]

In [25]:
# Funcion para cargar los json a una serie de pandas
def expandir_diccionario(diccionario):
    return pd.Series(diccionario)

In [26]:
# Se concatenas las series al df original
df_expandido = pd.concat([df_atributes, df_atributes['attributes'].apply(expandir_diccionario)], axis=1)
df_expandido.drop('attributes', axis=1, inplace=True)     # Se elimina la columna de MISC
df_atributes= df_expandido

In [27]:
df_atributes = df_atributes[['business_id','RestaurantsDelivery','OutdoorSeating','BusinessAcceptsCreditCards','GoodForKids',
                             'RestaurantsPriceRange2','RestaurantsTakeOut','RestaurantsReservations','HasTV']]

In [28]:
df_atributes= df_atributes.fillna('None')     # Se imputan los valores nulos a 'None'

In [29]:
df_atributes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7879 entries, 0 to 15275
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   business_id                 7879 non-null   object
 1   RestaurantsDelivery         7879 non-null   object
 2   OutdoorSeating              7879 non-null   object
 3   BusinessAcceptsCreditCards  7879 non-null   object
 4   GoodForKids                 7879 non-null   object
 5   RestaurantsPriceRange2      7879 non-null   object
 6   RestaurantsTakeOut          7879 non-null   object
 7   RestaurantsReservations     7879 non-null   object
 8   HasTV                       7879 non-null   object
dtypes: object(9)
memory usage: 615.5+ KB


In [30]:
df_atributes.head()

,business_id,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,GoodForKids,RestaurantsPriceRange2,RestaurantsTakeOut,RestaurantsReservations,HasTV
0,MTSW4McQd7CbVtyjqoe9mw,False,False,False,None,1,True,None,None
2,tMkwHmWFUEXrC9ZduonpTg,None,False,True,False,None,None,True,False
3,2xVsWBNFwZOxIOdd9Mwnww,False,True,True,True,2,True,None,True
6,ABxoFuzZy5mqQ8C5FJJajQ,None,None,None,None,None,None,None,None
7,uI9XODGY_2_ieTE6xJ0myw,False,True,True,True,2,True,True,True


In [31]:
#Borrar Columnas
df_business= df_business.drop(['address','postal_code', 'review_count', 'is_open', 'attributes','categories', 'hours','state','city'], axis=1)
df_business= df_business[['business_id','name', 'ciudad', 'estado', 'latitude', 'longitude', 'stars']]
df_business= df_business.rename(columns={'ciudad': 'city','estado': 'state'}) # cambiar nombre de las columnas
df_business['price'] = "SIN DATO"
df_business['platform'] = 2
df_business = df_business.reset_index(drop=True)

In [32]:
df_business.head(3)

,business_id,name,city,state,latitude,longitude,stars,price,platform
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,Pennsylvania,39.955505,-75.155564,4.0,SIN DATO,2
1,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,Nashville,Tennessee,36.159886,-86.773197,4.0,SIN DATO,2
2,2xVsWBNFwZOxIOdd9Mwnww,Cheeseburger In Paradise,Exton,Pennsylvania,40.029962,-75.630607,2.5,SIN DATO,2


##### Procedimiento para dividir un archivo .Json muy grande para ser procesado por partes por limites de memoria 

In [ ]:
def dividir_json_grande(archivo_entrada, num_partes):
    # Contar el número total de líneas/objetos en el archivo JSON
    with open(archivo_entrada, 'r', encoding='utf-8') as archivo:
        total_lineas = sum(1 for _ in archivo)
    
    # Calcular el tamaño de cada parte
    tamaño_parte = total_lineas // num_partes
    if total_lineas % num_partes != 0:
        tamaño_parte += 1

    # Leer y dividir el archivo en partes
    with open(archivo_entrada, 'r', encoding='utf-8') as archivo:
        for i in range(num_partes):
            archivo_salida = f'parte_{i + 1}.json'
            with open(archivo_salida, 'w', encoding='utf-8') as salida:
                salida.write('[')  # Comenzar un array JSON
                for j in range(tamaño_parte):
                    linea = archivo.readline()
                    if not linea:
                        break
                    salida.write(linea.rstrip())
                    if j < tamaño_parte - 1 and not archivo.tell() == total_lineas:
                        salida.write(',')
                salida.write(']')  # Terminar el array JSON
            # Si el archivo original ha terminado, salir del bucle
            if not linea:
                break

# Usar la función para dividir el archivo 'archivo.json' en 10 partes
dividir_json_grande('D:/PROYECTO FINAL/Yelp/review.json', 20)